In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
568,2024-09-30,Alemanha Bbl,15:00,Braunschweig,Ludwigsburg,2.81,1.44,162.5,1.87,1.91,4.5,1.99,1.80,GzFbFu0k,0.355872,0.694444,0.534759,0.523560,0.050316,0.6,0.6,NaN,NaN,206.102,76.830760,0.372780,195.522,76.718856,0.392380,197.334,156.288,304.20,136.08,1.0,0.0,0.0,0.0,0.455876,0.014965,0.070897,2.36,0.472,3.834746,0.515204,0.6,0.084796,-1.53,-0.306,-1.437908,0.594033,0.5,-0.094033
569,2024-09-30,Croácia Premijer Liga,15:15,Dinamo Zagreb,Zabok,1.25,3.69,165.5,1.84,1.86,-9.5,1.96,1.75,M5NLAW7b,0.800000,0.271003,0.543478,0.537634,0.071003,0.6,0.6,NaN,NaN,221.250,106.172727,0.479877,185.128,170.409789,0.920497,154.762,263.752,169.32,105.28,0.0,0.0,0.0,0.0,0.698518,0.007644,0.080050,-2.09,-0.418,-0.598086,0.000000,0.0,0.000000,8.77,1.754,1.533637,0.000000,0.0,0.000000
570,2024-09-30,Eslovênia Liga Otp Banka,12:30,Helios Domzale,Sencur,1.25,3.69,159.5,1.85,1.85,-9.5,1.95,1.76,Mybq5CAc,0.800000,0.271003,0.540541,0.540541,0.071003,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.698518,0.000000,0.072426,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
571,2024-09-30,Europa Liga Aba,14:00,Borac,Krka,1.30,3.34,158.5,1.84,1.86,-8.5,1.96,1.75,QVJN5LoL,0.769231,0.299401,0.543478,0.537634,0.068632,0.4,0.4,NaN,NaN,310.816,289.944108,0.932848,145.954,40.373749,0.276620,143.276,340.302,164.45,157.50,0.0,0.0,0.0,0.0,0.621766,0.007644,0.080050,-2.26,-0.452,-0.663717,0.000000,0.0,0.000000,-5.00,-1.000,-2.340000,0.000000,0.0,0.000000
572,2024-09-30,Filipinas Copa Dos Governadores,08:30,San Miguel Beermen,Converge FiberXers,1.24,3.87,204.5,1.80,1.86,-10.5,2.01,1.72,pOzfHStP,0.806452,0.258398,0.555556,0.537634,0.064850,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,135.66,304.95,0.0,0.0,0.0,0.0,0.727863,0.023184,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
573,2024-09-30,Romênia Divisão A,13:00,BC Timisoara,CSM Constanta,2.56,1.47,161.5,1.85,1.85,3.5,2.01,1.71,COkYlOel,0.390625,0.680272,0.540541,0.540541,0.070897,0.8,1.0,NaN,NaN,142.952,35.460810,0.248061,168.076,30.511625,0.181535,149.438,240.346,165.60,157.32,0.0,0.0,0.0,0.0,0.382504,0.000000,0.114049,-1.12,-0.224,-6.964286,0.000000,0.0,0.000000,0.79,0.158,2.974684,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
